<a href="https://colab.research.google.com/github/BRomans/IdMind/blob/main/autoencoder_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pytorch_lightning

     |████████████████████████████████| 686kB 7.0MB/s 
     |████████████████████████████████| 102kB 7.2MB/s 
     |████████████████████████████████| 276kB 15.2MB/s 
     |████████████████████████████████| 829kB 16.4MB/s 
     |████████████████████████████████| 1.3MB 33.3MB/s 
     |████████████████████████████████| 296kB 26.5MB/s 
     |████████████████████████████████| 143kB 31.2MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=6c1023ae0c4771d35a4fbe453cc233f8c4b177403ac77295db9bc4fed34b02d9
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=f4413794e2aab40a04a2a94f6f2f20acb39eee054533ec569f3be469cbe4b354
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3163 

In [3]:
import numpy as np
from matplotlib import pyplot as plt
from google.colab import drive
import os
import pickle
from pathlib import Path
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader

In [4]:
drive.mount("/content/drive")
filepath = "/content/drive/MyDrive/ml2-eeg-biometrics/eeg_dataset_right_hand_task_subset_9channels.csv"
# os.chdir('/content/drive/MyDrive/ml2-eeg-biometrics/prep-data')

Mounted at /content/drive


In [17]:
# df=pd.read_csv(filepath)
print(df.shape)
df.dropna(axis=1,how='all',inplace=True)
print(df.shape)

(20960781, 15)
(20960781, 14)


In [28]:
print(df.dtypes)

Participant     object
Date             int64
Run             object
Task             int64
Trial            int64
F3             float64
F4             float64
FC3            float64
FC4            float64
C3             float64
Cz             float64
C4             float64
CP3            float64
CP4            float64
dtype: object


In [42]:
# Create a train-validation-test split.
# For testing we want to take a separate person entirely (let's take S12) - this constitutes 5% of the total dataset.
# For validation we want to keep it equal across participants and sessions. We'll take 1 run per session for each of the participants. 
# # This is one sixth of the remaining data which equates to 15.8% of the total dataset. Thus, we keep 79.2% for training data.

drop_cols = ['Participant','Date', 'Run', 'Task', 'Trial']
# Convert Partipant ID to an integer column 'Target' so that pytorch can handle it.

df['Target'] = df['Participant'].astype('category').cat.codes.values

test_df = df[df['Participant']=='S12'].drop(drop_cols, axis=1)
print("test_df shape:", test_df.shape)

# Run5 looks most complete in the data.
valid_df = df[df['Run']=='Run5'].drop(drop_cols, axis=1)
print("valid_df shape:", valid_df.shape)

# Take the rest for training data
train_df = df[~ ((df['Run']=='Run5') & (df['Participant']=='S12'))].drop(drop_cols, axis=1)
print("train_df shape:", train_df.shape)

test_df shape: (1047500, 10)
valid_df shape: (3425000, 10)
train_df shape: (20785781, 10)


In [46]:
x_test = torch.tensor(test_df.drop('Target', axis=1).values)
y_test = torch.tensor(test_df['Target'].values)

x_valid = torch.tensor(valid_df.drop('Target', axis=1).values)
y_valid = torch.tensor(valid_df['Target'].values)

x_train = torch.tensor(train_df.drop('Target', axis=1).values)
y_train = torch.tensor(train_df['Target'].values)

In [ ]:
# df = pd.read_csv('File_explaination.csv')
# print(df)
# files = df.get('Filename')
# print(files[8384])

In [ ]:
# class RightHandTrain(Dataset):

#   def __init__(self, train_split):
#     df = pd.read_csv('File_explaination.csv')
#     self.files = df.get('Filename')
#     self.train_split = train_split
#     self.X = [None] * int(len(self.files)*self.train_split)

#   def __len__(self):
#     return int(len(self.files)*self.train_split)

#   def __getitem__(self, idx):
#     if self.X[idx] is None:
#       self.X[idx] = torch.load(self.files[idx])

#     return self.X[idx]

# class RightHandVal(Dataset):

#   def __init__(self, val_split):
#     df = pd.read_csv('File_explaination.csv')
#     self.files = df.get('Filename')
#     self.val_split = val_split
#     self.X = [None] * (int(len(self.files)*self.val_split)-1)

#   def __len__(self):
#     return int(len(self.files)*self.val_split) -1

#   def __getitem__(self, idx):
#     idx += int(len(self.files) * (1 - self.val_split)) +1 
#     if self.X[idx] is None:
#       self.X[idx] = torch.load(self.files[idx])

#     return self.X[idx]

In [ ]:
# ds_train = RightHandTrain(0.6)
# ds = RightHandTrain(0.6)
# #x_train, x_test = train_test_split(ds)
# len(ds)


In [ ]:
# x = torch.load('./A4|0|0|0.pt')
# x = x.view(1,29,2500)
# x = x.float()
# print(x.size())

In [ ]:
# class Autoencoder(pl.LightningModule):

#   def __init__(self):
#     super().__init__()
      
#     # encoder layers 
#     self.conv1 = nn.Conv1d(29, 50, 5)  
#     self.conv2 = nn.Conv1d(50, 20, 5)
#     self.conv3 = nn.Conv1d(20, 5, 5)

#     # decoder layers
#     self.t_conv1 = nn.ConvTranspose1d(5, 20, 5)
#     self.t_conv2 = nn.ConvTranspose1d(20, 50, 5)
#     self.t_conv3 = nn.ConvTranspose1d(50, 29, 5)

#     # pooling layer to reduce dims by two
#     #self.pool = nn.MaxPool1d(2)

#     # activation function
#     self.act = nn.LeakyReLU(0.1)

#     self.loss = nn.MSELoss()

#   def forward(self, x):
#     # Encode  
#     x = self.act(self.conv1(x))
#     x = self.act(self.conv2(x))
#     x = self.act(self.conv3(x))

#     # Decode
#     x = self.act(self.t_conv1(x))
#     x = self.act(self.t_conv2(x))
#     x = self.act(self.t_conv3(x))
#     return x

#   def training_step(self, batch, batch_idx):
#     x = batch
#     x_hat = self(x)
#     loss = self.loss(x_hat, x)
#     return loss

#   def configure_optimizers(self):
#     return torch.optim.Adam(self.parameters(), lr=0.02)

In [ ]:
# ae = Autoencoder()
# print(ae(x).size())
